In [2]:
# No.1
# 依存関係のセルです。セル内をクリックして、Ctrl + Enterを押し、左側に緑のチェックがつくまでお待ちください
!pip install binance-connector pandas ccxt pybit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.4 MB/s 
     |████████████████████████████████| 3.0 MB 9.8 MB/s 
     |████████████████████████████████| 473 kB 29.2 MB/s 
     |████████████████████████████████| 3.1 MB 42.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 396 kB 51.9 MB/s 
     |████████████████████████████████| 4.0 MB 40.2 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 251 kB 56.5 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 271 kB 56.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 291 kB 52.8 MB/s 
     |████████████████████████████████| 14

In [9]:
# No.2
#関数のセルです。Ctrl + Enterを押して、左側にチェックがつくまでお待ちください
def poloniex_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='BTC'):
      """
      poloniexのAPI使用
      USDTBTC専用の関数です。
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      import requests
      import pandas as pd
      from datetime import datetime
      import calendar


      startTime = calendar.timegm(datefrom.utctimetuple())
      endTime = calendar.timegm(dateto.utctimetuple())

      payload = {'command':'returnChartData',
                'currencyPair':'USDT_BTC',
                'start' : str(startTime),
                'end' : str(endTime),
                'period': 86400
                }


      url = "https://poloniex.com/public"
      r = requests.get(url, params=payload)
      df = pd.DataFrame(r.json(),columns=['date', 'open', 'close', 'high', 'low', 'volume'])
      df = df.rename(columns={'date':'open_time'})
      df_marketdata_polonix = df.drop(columns=['volume'])
      df_marketdata_polonix['open_time'] = df_marketdata_polonix['open_time'].map(lambda x: datetime.fromtimestamp(x).strftime('%Y/%m/%d'))
      return df_marketdata_polonix

def bitfinex_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='BTC'):
      """
      BITFINEXのAPI使用
      CHSBのみに使用する関数です
      CHSBのローソク足をBTCに換算します
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      import requests
      import pandas as pd
      import calendar
      from datetime import datetime

      if base_symbol == 'CHSB':
        symbol = f't{base_symbol}:{quote_symbol}'
      elif base_symbol == 'BTC':
        symbol = f't{base_symbol}{quote_symbol}'
      elif base_symbol == 'XLM':                         # 追加コメント　 この行を追加しました
        symbol = f't{base_symbol}{quote_symbol}'         # 追加コメント　 この行　を追加しました
      else:
        symbol = f't{base_symbol}{quote_symbol}''tXLMBTC'

      url =  f'https://api-pub.bitfinex.com/v2/candles/trade:1D:{symbol}/hist'


      start = calendar.timegm(datefrom.utctimetuple()) * 1000
      end = calendar.timegm(dateto.utctimetuple()) * 1000

      payload = {'start' : str(start),
                  'end' : str(end), 
                  'sort': 1,
                  'limit': 1000}

      r = requests.get(url,params=payload)

      df = pd.DataFrame(r.json(),columns=['open_time', 'open', 'close', 'high', 'low', 'volume'])
      df_marketdata_bitfinex= df.drop(columns=['volume'])
      df_marketdata_bitfinex['open_time'] = df_marketdata_bitfinex['open_time'].map(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y/%m/%d'))
      return df_marketdata_bitfinex

def Binance_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='BTC'):
      """
      BinanceのAPI使用
      デフォルトで対BTCに換算するする関数です。
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      from binance.spot import Spot
      import calendar
      from pandas import pandas as pd
      from datetime import datetime

      timeframe = '1d'
      client = Spot()
      symbol = f'{base_symbol}{quote_symbol}'
      presision = client.exchange_info(symbol = symbol)['symbols'][0]['quoteAssetPrecision']
      startTime = calendar.timegm(datefrom.utctimetuple()) * 1000
      endTime = calendar.timegm(dateto.utctimetuple()) * 1000

      cdls = client.klines(symbol,timeframe,startTime=startTime, endTime=endTime)

      df = pd.DataFrame(cdls,columns=['open_time', 'open', 'high', 'low', 'close', 'volume','close_time', 'quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
      df_marketdata = df.drop(columns=['volume','close_time', 'quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
      df_marketdata['open_time'] = df_marketdata['open_time'].map(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y/%m/%d'))

      return df_marketdata

def FTX_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='JPY'):
      """
      FTXのAPI使用
      デフォルトで対JPYに換算するする関数です。
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      import ccxt
      from pprint import pprint
      import pandas as pd
      from datetime import datetime
      import calendar
      symbol = f'{base_symbol}/{quote_symbol}'
      startTime = calendar.timegm(datefrom.utctimetuple()) * 1000

      dt = dateto - datefrom
      limit = dt.days + 1

      exchange = ccxt.ftx()
      candles = exchange.fetchOHLCV(symbol=symbol,timeframe='1d',limit=limit, since = startTime)
      df = pd.DataFrame(candles,columns=['open_time', 'open', 'high', 'low', 'close', 'volume'])
      df_marketdata_ftx= df.drop(columns=['volume'])
      df_marketdata_ftx['open_time'] = df_marketdata_ftx['open_time'].map(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y/%m/%d'))

      return df_marketdata_ftx


def poloniex_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='BTC'):
      """
      poloniexのAPI使用
      USDTBTC専用の関数です。
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      import requests
      import pandas as pd
      from datetime import datetime
      import calendar


      startTime = calendar.timegm(datefrom.utctimetuple())
      endTime = calendar.timegm(dateto.utctimetuple())

      payload = {'command':'returnChartData',
                'currencyPair':'USDT_BTC',
                'start' : str(startTime),
                'end' : str(endTime),
                'period': 86400
                }


      url = "https://poloniex.com/public"
      r = requests.get(url, params=payload)
      df = pd.DataFrame(r.json(),columns=['date', 'open', 'close', 'high', 'low', 'volume'])
      df = df.rename(columns={'date':'open_time'})
      df_marketdata_polonix = df.drop(columns=['volume'])
      df_marketdata_polonix['open_time'] = df_marketdata_polonix['open_time'].map(lambda x: datetime.fromtimestamp(x).strftime('%Y/%m/%d'))
      return df_marketdata_polonix

def coingecko_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='JPY'):
      """
      coingeckoのAPI使用
      デフォルトで対JPYに換算するする関数です。
      引数　ベース通貨　開始日　終了日　換算先通貨
      返り値　pandas.DataFrame
      """
      import requests
      import pandas as pd
      from datetime import datetime
      import calendar

      if base_symbol=='MV':
        base = 'gensokishis-metaverse'

      if base_symbol=='FITFI':
        base = 'step-app-fitfi'

      if base_symbol=='BIT':
        base = 'bitdao'

      if quote_symbol == 'JPY':
        quote = 'jpy'

      #datefromから昨日までが何日かを取得する
      td =  datetime.now() - datefrom
      days = 'max'
      url = f"https://api.coingecko.com/api/v3/coins/{base}/ohlc?vs_currency={quote}&days={days}"
      res = requests.get(url)

      df = pd.DataFrame(res.json(),columns=['time', 'open', 'high', 'low', 'close'])
      df_marketdata_gecko = df.rename(columns={'time':'open_time'})

      df_marketdata_gecko['open_time'] = df_marketdata_gecko['open_time'].map(lambda x: datetime.fromtimestamp(x/1000))
      df_marketdata_gecko = df_marketdata_gecko[(df_marketdata_gecko['open_time']>=datefrom) & (df_marketdata_gecko['open_time']<=dateto)]

      df_marketdata_gecko['open_time'] = df_marketdata_gecko['open_time'].map(lambda x: x.strftime('%Y/%m/%d'))

      return df_marketdata_gecko

def crypt_history_to_CSV(base_symbol:str,timeframe:str,start:str,end:str,quote_symbol:str='BTC', JPY_output:bool=False):
      """
      入力を受け取り、通貨ペアを各関数に振り分け、戻り値をCSVに書き込む関数です
      引数　ベース通貨　開始日　終了日　換算先通貨　JPY出力
      戻り値なし　ディレクトリにCSVが作成されます
      """
      import pandas as pd
      from datetime import datetime
      import calendar

      datefrom = datetime.strptime(start,'%Y/%m/%d')
      startTime = calendar.timegm(datefrom.utctimetuple()) * 1000
      dateto = datetime.strptime(end,'%Y/%m/%d')
      endTime = calendar.timegm(dateto.utctimetuple()) * 1000
      symbol = base_symbol+quote_symbol

      header_BTCmarket_info = ''
      header_additional_jpy_info = ''
      header_additional_jpy_market = ''
      presision = 8
      header_market = f'{base_symbol}/BTC換算情報'

      if base_symbol == 'BTC':
        
        #FTXが長期取得できないため、BitFenexに変更します
        #df_marketdata = FTX_market_dataframe(base_symbol,datefrom,dateto)

        #bitfinex_market_dataframe(base_symbol:str,datefrom:str,dateto:str,quote_symbol:str='BTC')

        # BTC JPY をbitfinexから取得する様に変更しました
        df_marketdata = bitfinex_market_dataframe(base_symbol,datefrom,dateto,'JPY')
        symbol = 'BTCJPY'
        header_BTCmarket_info = 'BitFenex'
        header_market = f'BTC/JPY換算情報'
        JPY_output = False

      elif base_symbol == 'CHSB' or base_symbol == 'XLM': # 追加コメント　 or base_symbol == 'XLM'　を追加しました

        df_marketdata = bitfinex_market_dataframe(base_symbol,datefrom,dateto)
        header_BTCmarket_info = 'bitfinex'

      elif base_symbol == 'SHIB':
        
        df_marketdata = Binance_market_dataframe(base_symbol,datefrom,dateto,'USDT')
        df_BTCUSDT = Binance_market_dataframe('BTC',datefrom,dateto,'USDT')

        df_marketdata['open'] = df_marketdata['open'].astype('float64')/df_BTCUSDT['close'].astype('float64')
        df_marketdata['close'] = df_marketdata['close'].astype('float64')/df_BTCUSDT['close'].astype('float64')
        df_marketdata['high'] = df_marketdata['high'].astype('float64')/df_BTCUSDT['close'].astype('float64')
        df_marketdata['low'] = df_marketdata['low'].astype('float64')/df_BTCUSDT['close'].astype('float64')
        header_BTCmarket_info = 'binance'

      elif base_symbol == 'MV' or base_symbol == 'FITFI' or  base_symbol == 'BIT':

        df_marketdata = coingecko_market_dataframe(base_symbol,datefrom,dateto)
        header_BTCmarket_info = ''
        JPY_output = False
        header_additional_jpy_info = 'BTC/JPY換算情報'
        header_additional_jpy_market = 'coingecko'

      else:

        df_marketdata = Binance_market_dataframe(base_symbol,datefrom,dateto)
        header_BTCmarket_info = 'Binance'

      if JPY_output:

        header_additional_jpy_info = 'BTC/JPY換算情報'
        header_additional_jpy_market = 'bitfenex 終値'
        df_marketdata_btcjpy = bitfinex_market_dataframe('BTC',datefrom,dateto,'JPY')
        df_marketdata['open'] = df_marketdata['open'].astype('float64')*df_marketdata_btcjpy['close'].astype('float64')
        df_marketdata['close'] = df_marketdata['close'].astype('float64')*df_marketdata_btcjpy['close'].astype('float64')
        df_marketdata['high'] = df_marketdata['high'].astype('float64')*df_marketdata_btcjpy['close'].astype('float64')
        df_marketdata['low'] = df_marketdata['low'].astype('float64')*df_marketdata_btcjpy['close'].astype('float64')
        symbol = base_symbol + 'JPY'


      df_header = pd.DataFrame([[symbol,'','開始日',datefrom.strftime('%Y/%m/%d'),'終了日',dateto.strftime('%Y/%m/%d')],
                                [header_market,header_BTCmarket_info],
                                [header_additional_jpy_info, header_additional_jpy_market ],
                                ['年月日','始値','高値','安値','終値']])

      filename = f"{symbol}_{datefrom.strftime('%Y%m%d')}_{dateto.strftime('%Y%m%d')}.csv"

      with open(rf"{filename}",mode = 'w', encoding='utf-8-sig') as f:
          df_header.to_csv(f,index = False, header=False,encoding='utf-8-sig')

      with open(rf"{filename}", mode = 'a',encoding='utf-8-sig') as f:
          df_marketdata.to_csv(f, header=f.tell()==0,index = False, float_format=f'%.{presision}f')

In [11]:
# No.3
# 関数を実行するセルです。中のデータを編集して、Ctrl + Enterを押してください
# なお、関数セルが先に実行されていないと機能しません

coins = ['BTC','ETH','UST','NEXO','DOT','XLM','XRP','BAT','DOGE','SAND','MANA','LSK','SOL','APE','CHSB','SHIB','MV','BIT','FITFI','ADA']
coins = ['BTC']
#'yyyy/mm/dd'形式で指定します
startdate = '2021/6/1'
enddate = '2022/5/31'

#暗号通貨を一括で処理します
# JPY_outputをTrueにすると、全ての通貨が日本円に換算されます。
# JPY_outputをFalseにすると、ほぼすべての通貨がBTCに換算されますが、JPYやUSDTに換算される通貨もあります

for crypt in coins:
  print(crypt)
  crypt_history_to_CSV(crypt,'1d',startdate,enddate,JPY_output = False)




BTC
